In [ ]:
from collections import defaultdict

import numpy as np

from astropy.table import Table 
from astrowidgets import ImageWidget
from bqplot import pyplot as plt
import ipywidgets as ipw

from stellarphot.differential_photometry import catalog_search

## Settings

Enter two things below:

+ the table with the latest relative flux calculation.
+ the aperture location file

In [ ]:
ta = Table.read('relative_flux-kelt-1.fits')
aps = Table.read('aperture_locations.fits')
aps

In [ ]:
vsx = aps[aps['marker name'] == 'VSX']
vsx

In [ ]:
vsx_ids = vsx['id']
vsx_dropdown = [(f'V{v}', v) for v in vsx_ids]

new_cols = defaultdict(list)

for row in vsx:
    foo = catalog_search(row['coord'], (10, 10), 'B/vsx/vsx', 
                         ra_column='RAJ2000', dec_column='DEJ2000', 
                         radius=0.01, clip_by_frame=False)
    this_star = foo[0]
    for data in ['Name', 'Type', 'V', 'Period', 'max', 'min']:
        new_cols[data].append(this_star[data])

for k, v in new_cols.items():
    vsx.add_column(v, name=k)

In [ ]:
fig = plt.figure(animation_duration=1000)
fig.layout.width = '100%'
sc = plt.scatter([0, 1, 2, 6, 4], [0, 0, 0, 0, 0])
#plt.show()

In [ ]:
def update_plot(change):
    new_id = change['new']
    new_y_data = ta['relative_flux'][ta['id'] == new_id]
    sc.y = new_y_data / np.nanmean(new_y_data)
    sc.x = ta['BJD'][ta['id'] == new_id]

In [ ]:
hbo = ipw.HBox(layout=ipw.Layout(width='100%'))
ddown = ipw.Dropdown(options=vsx_dropdown)
hbo.children = [fig, ddown]

# Draw the light curve for the first variable in case there is only 1
change = {'new': vsx_dropdown[0][1]}
update_plot(change)

In [ ]:
ddown.observe(update_plot, names='value')

In [ ]:
vsx

In [ ]:
hbo